# Задание
### Преподаватель: 
Олег Булыгин, Даниил Корбут, Наталья Баданина

### Цель:
Применить на практике алгоритмы по автоматической оптимизации параметров моделей машинного обучения.

### Описание задания:
В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с [сайта](https://www.kaggle.com/fedesoriano/heart-failure-prediction). Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

### Этапы работы:

1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)

2. Подготовьте датасет к обучению моделей:  
a) Категориальные переменные переведите в цифровые значения. Можно использовать [pd.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html), [preprocessing.LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). Старайтесь не использовать для этой задачи циклы.

3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

4. Обучите модель логистической регрессии с параметрами по умолчанию.

5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:
<code>    cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])</code>

6. Оптимизируйте 3-4 параметра модели:  
a) Используйте GridSearchCV.  
b) Используйте RandomizedSearchCV.  
c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.  
d) Повторите п. 5 после каждого итогового изменения параметров.  

7. Сформулируйте выводы по проделанной работе:  
a) Сравните метрики построенных моделей.  
b) *Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».  

### Для получения зачета по этому домашнему заданию минимально необходимо:
- обучить одну модель классификации;
- оптимизировать параметры, используя метод из п. 6a;
- вывести значения метрик.

### Результат:
- Получены знания по оптимизации параметров.

### Форма выполнения:
- ссылка на Jupyter Notebook, загруженный на GitHub;
- ссылка на Google Colab;
- файл с расширением .ipynb.

### Инструменты:
- Jupyter Notebook/Google Colab;
- GitHub;
- сайт с данными для обучения моделей.

### Срок выполнения: 
дедлайн приема решений на проверку

### *Рекомендации к выполнению:
- Текст оформляйте в отдельной ячейке Jupyter Notebook/Google Colab в формате markdown.
- У графиков должен быть заголовок, подписи осей, легенда (опционально). Делайте графики бОльшего размера, чем стандартный вывод, чтобы увеличить читабельность.
- Убедитесь, что по ссылкам есть доступ на чтение/просмотр.
- Убедитесь, что все ячейки в работе выполнены и можно увидеть их вывод без повторного запуска.

### 1. Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)

In [1]:
import numpy as np
import pandas as pd # Для работы с данными
import scipy.stats # При работе со статистикой
import seaborn as sns
import matplotlib.pyplot as plt  # Библиотека для визуализации результатов
%matplotlib inline

In [2]:
data = pd.read_csv('heart.csv')
print(data.info())
print(data.describe())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB
None
              Age   RestingBP  Cholesterol   FastingBS       MaxHR  \
count  918.000000  918.000000   918.000000  918.000000  918.000000   
mean    53.510893  132.396514   198.799564    0.233115  1

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


### 2. Подготовьте датасет к обучению моделей.  
a) Категориальные переменные переведите в цифровые значения. Можно использовать [pd.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html), [preprocessing.LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). Старайтесь не использовать для этой задачи циклы. 

In [3]:
# Три колонки преобразуем с помощью cols_to_dummies
cols_to_dummies = ['ChestPainType', 'RestingECG', 'ST_Slope']
data2 = pd.get_dummies(data[cols_to_dummies], columns=cols_to_dummies).copy()
data2.describe()

,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,0.540305,0.188453,0.221133,0.050109,0.204793,0.601307,0.193900,0.068627,0.501089,0.430283
std,0.498645,0.391287,0.415236,0.218289,0.403770,0.489896,0.395567,0.252957,0.500271,0.495386
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
# Две колонки с бинарными категориями преобразуем с помощью LabelEncoder
from sklearn import preprocessing
le_Sex = preprocessing.LabelEncoder()
le_Sex.fit(data.Sex.unique())
data_Sex_int = le_Sex.transform(data.Sex)
# смотрим первые сто элементов
data_Sex_int[0:100]

array([1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1])

In [5]:
# Вторая колонка
le_ExerciseAngina = preprocessing.LabelEncoder()
le_ExerciseAngina.fit(data.ExerciseAngina.unique())
data_ExerciseAngina_int = le_ExerciseAngina.transform(data.ExerciseAngina)
# смотрим первые сто элементов
data_ExerciseAngina_int[0:100]

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0])

In [6]:
# Соберем датасет признаков и выделим серию с целевой переменной
X = pd.concat([data[['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']], 
               data2, 
               pd.DataFrame({'Sex2': data_Sex_int}), 
               pd.DataFrame({'ExerciseAngina2': data_ExerciseAngina_int})], axis=1)
y = data['HeartDisease'].copy()
print(f'Целевая колонка (первые 5 строк):\n{y[0:5]}')
X.head()

Целевая колонка (первые 5 строк):
0    0
1    1
2    0
3    1
4    0
Name: HeartDisease, dtype: int64


,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up,Sex2,ExerciseAngina2
0,40,140,289,0,172,0.0,0,1,0,0,0,1,0,0,0,1,1,0
1,49,160,180,0,156,1.0,0,0,1,0,0,1,0,0,1,0,0,0
2,37,130,283,0,98,0.0,0,1,0,0,0,0,1,0,0,1,1,0
3,48,138,214,0,108,1.5,1,0,0,0,0,1,0,0,1,0,0,1
4,54,150,195,0,122,0.0,0,0,1,0,0,1,0,0,0,1,1,0


### 3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [7]:
# Грузим модуль
from sklearn.model_selection import train_test_split
# Разделяем:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21) 
# смотрим, как прошло разбиение
print(X.shape)
print(X_train.shape)   

(918, 18)
(734, 18)


### 4. Обучите модель логистической регрессии с параметрами по умолчанию.

In [8]:
# Вначале - логистическая регрессия
from sklearn.linear_model import LogisticRegression

# Создаем с параметрами по умолчанию 
data_logistic_regr = LogisticRegression(random_state=42)

data_logistic_regr.fit(X_train, y_train)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [11]:
from sklearn.preprocessing import MinMaxScaler

MMScaler = MinMaxScaler()
X_train_scale = MMScaler.fit_transform(X_train)
#print(X_train_scale)

# Создаем с параметрами по умолчанию 
data_logistic_regr_scale = LogisticRegression(random_state=42)

data_logistic_regr_scale.fit(X_train_scale, y_train)

LogisticRegression(random_state=42)

### 5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:  
<code>cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])</code>

In [13]:
from sklearn.model_selection import cross_validate
X_scale = MinMaxScaler().fit_transform(X)
scores = cross_validate(data_logistic_regr_scale, X_scale, y, cv=10, scoring=['accuracy', 'recall', 'precision', 'f1'])
scores

{'fit_time': array([0.14699173, 0.07499599, 0.04199791, 0.06499553, 0.04299712,
        0.06099677, 0.03699732, 0.05499482, 0.04399633, 0.04499674]),
 'score_time': array([0.00699854, 0.00699925, 0.00799894, 0.01299977, 0.00699925,
        0.01300073, 0.00800061, 0.00699997, 0.00799918, 0.0179987 ]),
 'test_accuracy': array([0.82608696, 0.93478261, 0.85869565, 0.93478261, 0.88043478,
        0.83695652, 0.88043478, 0.82608696, 0.72527473, 0.76923077]),
 'test_recall': array([0.76470588, 0.92156863, 0.82352941, 0.96078431, 0.98039216,
        0.96078431, 0.98039216, 0.82352941, 0.78      , 0.72      ]),
 'test_precision': array([0.90697674, 0.95918367, 0.91304348, 0.9245283 , 0.83333333,
        0.79032258, 0.83333333, 0.85714286, 0.73584906, 0.8372093 ]),
 'test_f1': array([0.82978723, 0.94      , 0.86597938, 0.94230769, 0.9009009 ,
        0.86725664, 0.9009009 , 0.84      , 0.75728155, 0.77419355])}

### 6. Оптимизируйте 3-4 параметра модели:  
a) Используйте GridSearchCV.  

In [14]:
from sklearn.model_selection import GridSearchCV

In [115]:
# Параметры, вариации которых я пытался смотреть
param_grid = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [10, 20, 50, 70, 100, 200],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'c': [0.5, 0.75, 1, 1.5, 3]
}

# Здесь я несколько раз переопределял различные комбинации этих параметров, в результате чего смог увидеть,
# что параметр C никак не влиял на результат, penalty = none было хорошим выбором, возможно потому, что данные нормализованы
# max_iter =20 оказывается достаточным, solver=saga
param_grid = {
    'penalty': ['l2', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [10, 20, 50, 70, 100, 200],
    'C': [0.5, 0.75, 1, 1.5, 3]
}


In [116]:
# инициализируем grid
grid = GridSearchCV(LogisticRegression(random_state=42), param_grid, cv=10, scoring='accuracy')

In [117]:
import warnings
warnings.filterwarnings('ignore')
grid.fit(X_train_scale, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(random_state=42),
             param_grid={'C': [0.5, 0.75, 1, 1.5, 3],
                         'max_iter': [10, 20, 50, 70, 100, 200],
                         'penalty': ['l2', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [118]:
#for k in grid.cv_results_:
#    print(k, ":", grid.cv_results_[k])

print('mean_test_score\n', grid.cv_results_['mean_test_score'])
print('\nrank_test_score\n', grid.cv_results_['rank_test_score'])

mean_test_score
 [0.85816364 0.86088486 0.86223621 0.85814513 0.85953351 0.86358756
 0.85407257        nan 0.86084783 0.86088486 0.85816364 0.85816364
 0.86223621 0.85816364 0.85953351 0.86358756 0.86358756        nan
 0.86088486 0.86223621 0.85816364 0.85816364 0.86223621 0.85816364
 0.85816364 0.86358756 0.86358756        nan 0.86358756 0.86358756
 0.85816364 0.85816364 0.86223621 0.85816364 0.85816364 0.86358756
 0.86358756        nan 0.86358756 0.86358756 0.85816364 0.85816364
 0.86223621 0.85816364 0.85816364 0.86358756 0.86358756        nan
 0.86358756 0.86358756 0.85816364 0.85816364 0.86223621 0.85816364
 0.85816364 0.86358756 0.86358756        nan 0.86358756 0.86358756
 0.86223621 0.86225472 0.86223621 0.85951499 0.86360607 0.86358756
 0.85407257        nan 0.86084783 0.86088486 0.86223621 0.86223621
 0.86223621 0.86223621 0.86223621 0.86358756 0.86358756        nan
 0.86088486 0.86223621 0.86223621 0.86223621 0.86223621 0.86223621
 0.86223621 0.86358756 0.86358756        nan 

In [119]:
print(grid.best_score_)
print(grid.best_estimator_)

0.8649759348389485
LogisticRegression(C=1.5, max_iter=10, random_state=42, solver='saga')


In [124]:
# Задаем параметры, которые кажутся оптимальными после использования grid
# Хотя код выше предложил в качестве оптимума max_iter = 10, ряд моих экспериментов показал, что 20 - лучше.
data_logistic_regr_scale_grid = LogisticRegression(random_state=42, C=1.5, penalty='none', solver='saga', max_iter=20)
data_logistic_regr_scale_grid.fit(X_train_scale, y_train)

scores_after_grid = cross_validate(data_logistic_regr_scale_grid, X_scale, y, cv=10, scoring=['accuracy', 'recall', 'precision', 'f1'])
scores_after_grid

{'fit_time': array([0.01399946, 0.01599813, 0.04899764, 0.02399898, 0.05299664,
        0.01899886, 0.0179975 , 0.02099872, 0.01699877, 0.01799726]),
 'score_time': array([0.03899789, 0.01100159, 0.0069983 , 0.0179987 , 0.01699877,
        0.01099944, 0.01200008, 0.0129993 , 0.00700068, 0.00799942]),
 'test_accuracy': array([0.82608696, 0.92391304, 0.85869565, 0.95652174, 0.86956522,
        0.84782609, 0.88043478, 0.82608696, 0.72527473, 0.79120879]),
 'test_recall': array([0.76470588, 0.90196078, 0.82352941, 0.96078431, 0.98039216,
        1.        , 0.98039216, 0.80392157, 0.76      , 0.72      ]),
 'test_precision': array([0.90697674, 0.95833333, 0.91304348, 0.96078431, 0.81967213,
        0.78461538, 0.83333333, 0.87234043, 0.74509804, 0.87804878]),
 'test_f1': array([0.82978723, 0.92929293, 0.86597938, 0.96078431, 0.89285714,
        0.87931034, 0.9009009 , 0.83673469, 0.75247525, 0.79120879])}

In [125]:
# Результаты оказались чуть-чуть лучше, чем с параметрами по умолчанию.
print(f'Метрика accuracy из умолчального эксперимента: {scores["test_accuracy"].mean()}')
print(f'Метрика accuracy из эксперимента c grid: {scores_after_grid["test_accuracy"].mean()}')

Метрика accuracy из умолчального эксперимента: 0.8472766364070713
Метрика accuracy из эксперимента c grid: 0.8505613951266126


### 6. Оптимизируйте 3-4 параметра модели:    
b) Используйте RandomizedSearchCV.

In [113]:
from sklearn.model_selection import RandomizedSearchCV

In [126]:
# Повторяю и здесь свои эксперименты, которые проводил с Grid,
# что параметр C никак не влиял на результат, penalty = none было хорошим выбором, возможно потому, что данные нормализованы
# max_iter =20 оказывается достаточным, solver=saga
param_rand_srch = {
    'penalty': ['l2', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [10, 20, 50, 70, 100, 200],
    'C': [0.5, 0.75, 1, 1.5, 3]
}


In [127]:
# инициализируем grid
rand_srch = RandomizedSearchCV(LogisticRegression(random_state=42), param_rand_srch, cv=10, scoring='accuracy')

In [128]:
import warnings
warnings.filterwarnings('ignore')
rand_srch.fit(X_train_scale, y_train)

RandomizedSearchCV(cv=10, estimator=LogisticRegression(random_state=42),
                   param_distributions={'C': [0.5, 0.75, 1, 1.5, 3],
                                        'max_iter': [10, 20, 50, 70, 100, 200],
                                        'penalty': ['l2', 'none'],
                                        'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear', 'sag',
                                                   'saga']},
                   scoring='accuracy')

In [129]:
print('mean_test_score\n', rand_srch.cv_results_['mean_test_score'])
print('\nrank_test_score\n', rand_srch.cv_results_['rank_test_score'])

mean_test_score
 [0.86360607 0.85953351 0.86358756 0.86223621 0.86223621 0.86358756
 0.86358756 0.86223621 0.86358756 0.86223621]

rank_test_score
 [ 1 10  3  6  6  2  3  6  3  6]


In [130]:
print(rand_srch.best_score_)
print(rand_srch.best_estimator_)

0.86360607182525
LogisticRegression(C=0.75, max_iter=10, random_state=42, solver='saga')


In [131]:
# Задаем параметры, которые кажутся оптимальными после использования RandomSearch
data_logistic_regr_scale_rand_srch = LogisticRegression(random_state=42, C=0.75, penalty='none', solver='saga', max_iter=10)
data_logistic_regr_scale_rand_srch.fit(X_train_scale, y_train)

scores_after_rand_srch = cross_validate(data_logistic_regr_scale_rand_srch, X_scale, y, cv=10, scoring=['accuracy', 'recall', 'precision', 'f1'])
scores_after_rand_srch

{'fit_time': array([0.00899839, 0.05299616, 0.01199865, 0.02599859, 0.0129993 ,
        0.02399778, 0.01399803, 0.01499915, 0.01299906, 0.01599836]),
 'score_time': array([0.00699973, 0.01099992, 0.00700068, 0.00799847, 0.01099944,
        0.01400042, 0.01099968, 0.00700045, 0.00699925, 0.01199937]),
 'test_accuracy': array([0.82608696, 0.93478261, 0.85869565, 0.94565217, 0.86956522,
        0.83695652, 0.88043478, 0.82608696, 0.72527473, 0.76923077]),
 'test_recall': array([0.76470588, 0.92156863, 0.82352941, 0.96078431, 0.98039216,
        0.98039216, 0.98039216, 0.82352941, 0.76      , 0.7       ]),
 'test_precision': array([0.90697674, 0.95918367, 0.91304348, 0.94230769, 0.81967213,
        0.78125   , 0.83333333, 0.85714286, 0.74509804, 0.85365854]),
 'test_f1': array([0.82978723, 0.94      , 0.86597938, 0.95145631, 0.89285714,
        0.86956522, 0.9009009 , 0.84      , 0.75247525, 0.76923077])}

### 7. Сформулируйте выводы по проделанной работе:  
a) Сравните метрики построенных моделей.  
b) *Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».

In [133]:
# Выводим рядом все три результата
print(f'Метрика accuracy из умолчального эксперимента: {scores["test_accuracy"].mean()}')
print(f'Метрика accuracy из эксперимента c grid: {scores_after_grid["test_accuracy"].mean()}')
print(f'Метрика accuracy из эксперимента c random search: {scores_after_rand_srch["test_accuracy"].mean()}')

Метрика accuracy из умолчального эксперимента: 0.8472766364070713
Метрика accuracy из эксперимента c grid: 0.8505613951266126
Метрика accuracy из эксперимента c random search: 0.8472766364070712


In [ ]:
# Фактически удалось чуть-чуть улучшить точность предсказания благодаря внимательному просмотру нескольких запусков

# grid с разными комбинациями параметров и установка пойманного в ходе тех экспериментов значения вопреки рекомендации
# Понимаю, что некоторые из использованных мною диапазонов значений - результат случайного выбора чисел. 
# По хорошему можно было бы добавить дополнительные числа для расширения диапазона экспериментов.


#### В работе по изучению ансамблирования были получены следующие результаты точности:
- Точность 0.8152, модель DecisionTreeClassifier
- Точность 0.8859, модель RandomForestClassifier
- Точность 0.8533, модель DecisionTreeClassifier, n_estimators=10
- Точность 0.8533, модель DecisionTreeClassifier, n_estimators=20
- Точность 0.8641, модель DecisionTreeClassifier, n_estimators=30
- Точность 0.8587, модель StackingClassifier with LogisticRegression, n_estimators=10
- Точность 0.7989, модель StackingClassifier with DecisionTreeClassifier, n_estimators=10
- Точность 0.8261, модель StackingClassifier with RandomForestClassifier, n_estimators=10

При этом в той работе LogisticRegression не использовалось, и видно, что RandomForestClassifier при использовании методов ансамблирования явно оказался более убедительным, чем Логистическая регрессия в текущем задании.